## Linking a dataset of real historical persons

In this example, we deduplicate a more realistic dataset. The data is based on historical persons scraped from wikidata. Duplicate records are introduced with a variety of errors introduced.

Note, as explained in the [backends topic guide](https://moj-analytical-services.github.io/splink/topic_guides/backends.html#sqlite), SQLite does not natively support string fuzzy matching functions such as `damareau-levenshtein` and `jaro-winkler` (as used in this example). Instead, these have been imported as python User Defined Functions (UDFs). One drawback of python UDFs is that they are considerably slower than native-SQL comparisons. As such, if you are hitting issues with large run times, consider switching to DuckDB (or some other backend).


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/sqlite/deduplicate_50k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev
# !pip install rapidfuzz

In [2]:
import pandas as pd

from splink import splink_datasets

pd.options.display.max_rows = 1000
# reduce size of dataset to make things run faster
df = splink_datasets.historical_50k.sample(5000)

In [3]:
from splink import SQLiteAPI
from splink.exploratory import profile_columns

db_api = SQLiteAPI()
profile_columns(
    df, db_api, column_expressions=["first_name", "postcode_fake", "substr(dob, 1,4)"]
)

alt.VConcatChart(...)

In [4]:
from splink import SQLiteAPI, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules =  [block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name")]

db_api = SQLiteAPI()

cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rule_creators=blocking_rules,
    db_api=db_api,
    link_type="dedupe_only",
    unique_id_column_name="unique_id",
)

alt.Chart(...)

In [5]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink import Linker

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name"),

    ],
    "comparisons": [
        ctl.NameComparison("first_name", fuzzy_thresholds=[0.9]).configure(
            term_frequency_adjustments=True
        ),
        ctl.NameComparison("surname", fuzzy_thresholds=[0.9]).configure(
            term_frequency_adjustments=True
        ),
        cl.DamerauLevenshteinAtThresholds("dob", [1, 2]).configure(
            term_frequency_adjustments=True
        ),
        cl.DamerauLevenshteinAtThresholds("postcode_fake", [1, 2]),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch(
            "occupation",
        ).configure(term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01,
}

linker = Linker(df, settings, database_api=db_api)

In [6]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.00013.
This means that amongst all possible pairwise record comparisons, one in 7,667.18 are expected to match.  With 12,497,500 total possible comparisons, we expect a total of around 1,630.00 matching pairs


In [7]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


In [8]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_names = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule, estimate_without_term_frequencies=True
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname


Iteration 1: Largest change in params was -0.435 in probability_two_random_records_match


Iteration 2: Largest change in params was -0.0222 in probability_two_random_records_match


Iteration 3: Largest change in params was -0.00727 in the m_probability of birth_place, level `All other comparisons`



EM converged after 3 iterations



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).


In [9]:
training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule, estimate_without_term_frequencies=True
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Iteration 1: Largest change in params was -0.315 in the m_probability of first_name, level `Exact match on first_name`


Iteration 2: Largest change in params was -0.0462 in the m_probability of first_name, level `Exact match on first_name`


Iteration 3: Largest change in params was -0.0104 in the m_probability of surname, level `Exact match on surname`


Iteration 4: Largest change in params was -0.00254 in the m_probability of surname, level `Exact match on surname`



EM converged after 4 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


The final match weights can be viewed in the match weights chart:


In [10]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [11]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [12]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key
0,30.737921,1.000000,Q6139106-4,Q6139106-5,james,james,2,0.026226,0.026226,43.192839,...,125.266581,2.594855,None,None,-1,NaN,NaN,1.000000,1.000000,0
1,-10.728443,0.000589,Q5545144-4,Q608545-9,george,george,2,0.032032,0.032032,43.192839,...,0.173283,1.000000,rugby union player,entomologist,0,0.013456,0.001922,0.084415,1.000000,0
2,35.606636,1.000000,Q4888351-1,Q4888351-2,benjamin,benjamin,2,0.003203,0.003203,43.192839,...,125.266581,4.757234,rugby union player,rugby union player,1,0.013456,0.013456,24.444413,2.793146,0
3,35.606636,1.000000,Q4888351-1,Q4888351-4,benjamin,benjamin,2,0.003203,0.003203,43.192839,...,125.266581,4.757234,rugby union player,rugby union player,1,0.013456,0.013456,24.444413,2.793146,0
4,36.866810,1.000000,Q1293322-1,Q1293322-4,edward,edward,2,0.018418,0.018418,43.192839,...,125.266581,2.195647,priest,priest,1,0.024606,0.024606,24.444413,1.527502,0


You can also view rows in this dataset as a waterfall chart as follows:


In [13]:

records_to_plot = df_e.to_dict(orient="records")
linker.waterfall_chart(records_to_plot, filter_nulls=False)

alt.LayerChart(...)

In [14]:
clusters = linker.cluster_pairwise_predictions_at_threshold(
    df_predict, threshold_match_probability=0.95
)

Completed iteration 1, root rows count 4


Completed iteration 2, root rows count 0


In [15]:
linker.cluster_studio_dashboard(
    df_predict,
    clusters,
    "dashboards/50k_cluster.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)

from IPython.display import IFrame

IFrame(src="./dashboards/50k_cluster.html", width="100%", height=1200)

In [16]:
linker.roc_chart_from_labels_column("cluster", match_weight_round_to_nearest=0.02)

alt.Chart(...)

In [17]:
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.999,
    include_false_negatives=False,
    include_false_positives=True,
).as_record_dict()
linker.waterfall_chart(records)

alt.LayerChart(...)

In [18]:
# Some of the false negatives will be because they weren't detected by the blocking rules
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.5,
    include_false_negatives=True,
    include_false_positives=False,
).as_record_dict(limit=50)

linker.waterfall_chart(records)

alt.LayerChart(...)